# 2. 自注意力机制 (Self-Attention)

在上一个教程中，我们学习了注意力机制的基本概念。现在我们将深入学习**自注意力机制**，这是Transformer架构的核心创新点。

## 2.1 什么是自注意力机制？

**传统注意力机制**：Query来自目标序列，Key和Value来自源序列
- 例如：机器翻译中，Query来自目标语言，Key和Value来自源语言

**自注意力机制**：Query、Key和Value都来自同一个序列
- 序列中的每个位置都可以关注序列中的所有位置（包括自己）
- 用于捕获序列内部的依赖关系

![自注意力机制示意图](images/self_attention.png)

### 核心思想：
让序列中的每个元素都能够"看到"整个序列，并学习到与其他元素的关系。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

# 设置随机种子和图表样式
torch.manual_seed(42)
np.random.seed(42)
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

print(f"PyTorch版本: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

## 2.2 自注意力的数学公式

自注意力机制的核心是通过线性变换将输入序列映射为Query、Key和Value：

$$Q = XW^Q$$
$$K = XW^K$$  
$$V = XW^V$$

然后计算自注意力：

$$\text{SelfAttention}(X) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

其中：
- $X \in \mathbb{R}^{n \times d}$ 是输入序列矩阵（n个位置，每个位置d维）
- $W^Q, W^K, W^V \in \mathbb{R}^{d \times d_k}$ 是可学习的权重矩阵
- $d_k$ 是Query和Key的维度

In [ ]:
class SelfAttention(nn.Module):
    """
    自注意力机制的实现
    """
    def __init__(self, d_model, d_k=None):
        super(SelfAttention, self).__init__()
        
        if d_k is None:
            d_k = d_model
            
        self.d_model = d_model
        self.d_k = d_k
        
        # 定义Q、K、V的线性变换层
        self.W_q = nn.Linear(d_model, d_k, bias=False)
        self.W_k = nn.Linear(d_model, d_k, bias=False)
        self.W_v = nn.Linear(d_model, d_k, bias=False)
        
        # 输出投影层
        self.W_o = nn.Linear(d_k, d_model)
        
    def forward(self, x, mask=None):
        """
        前向传播
        
        Args:
            x: 输入序列 [batch_size, seq_len, d_model]
            mask: 注意力掩码 [batch_size, seq_len, seq_len]
        
        Returns:
            output: 自注意力输出 [batch_size, seq_len, d_model]
            attention_weights: 注意力权重 [batch_size, seq_len, seq_len]
        """
        batch_size, seq_len, d_model = x.size()
        
        # 计算Q、K、V
        Q = self.W_q(x)  # [batch_size, seq_len, d_k]
        K = self.W_k(x)  # [batch_size, seq_len, d_k]
        V = self.W_v(x)  # [batch_size, seq_len, d_k]
        
        # 计算注意力分数
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        # scores shape: [batch_size, seq_len, seq_len]
        
        # 应用掩码（如果提供）
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        # 计算注意力权重
        attention_weights = F.softmax(scores, dim=-1)
        
        # 应用注意力权重
        context = torch.matmul(attention_weights, V)
        # context shape: [batch_size, seq_len, d_k]
        
        # 输出投影
        output = self.W_o(context)
        # output shape: [batch_size, seq_len, d_model]
        
        return output, attention_weights

# 测试自注意力模块
d_model = 64
seq_len = 8
batch_size = 2

# 创建随机输入
x = torch.randn(batch_size, seq_len, d_model)

# 创建自注意力层
self_attention = SelfAttention(d_model)

# 前向传播
output, attention_weights = self_attention(x)

print(f"输入形状: {x.shape}")
print(f"输出形状: {output.shape}")
print(f"注意力权重形状: {attention_weights.shape}")
print(f"\n注意力权重每行的和（应该为1）: {attention_weights[0].sum(dim=-1)}")

## 2.3 可视化自注意力权重

让我们可视化自注意力权重矩阵，理解序列中不同位置之间的关注关系：

In [ ]:
def visualize_self_attention(attention_weights, tokens=None, title="自注意力权重矩阵"):
    """
    可视化自注意力权重
    
    Args:
        attention_weights: 注意力权重矩阵 [seq_len, seq_len]
        tokens: 可选的token列表，用于标注坐标轴
        title: 图表标题
    """
    # 如果是批次数据，取第一个样本
    if attention_weights.dim() == 3:
        weights = attention_weights[0].detach().numpy()
    else:
        weights = attention_weights.detach().numpy()
    
    plt.figure(figsize=(10, 8))
    
    # 创建热力图
    if tokens is not None:
        sns.heatmap(weights, annot=True, cmap='Blues', fmt='.3f',
                    xticklabels=tokens, yticklabels=tokens)
    else:
        sns.heatmap(weights, annot=True, cmap='Blues', fmt='.3f',
                    xticklabels=[f'位置{i}' for i in range(weights.shape[1])],
                    yticklabels=[f'位置{i}' for i in range(weights.shape[0])])
    
    plt.title(title)
    plt.xlabel('被关注的位置 (Key)')
    plt.ylabel('查询位置 (Query)')
    plt.tight_layout()
    plt.show()

# 可视化刚才计算的自注意力权重
visualize_self_attention(attention_weights, title="自注意力权重可视化")

## 2.4 实际例子：句子中的自注意力

让我们通过一个具体的句子例子来理解自注意力机制如何工作：

In [ ]:
# 创建一个简单的词嵌入模拟
def create_word_embeddings():
    """
    创建简单的词嵌入，用于演示
    """
    # 定义一个小词汇表
    vocab = ['the', 'cat', 'sat', 'on', 'mat', 'big', 'small']
    vocab_size = len(vocab)
    embed_dim = 16
    
    # 创建词嵌入层
    embedding = nn.Embedding(vocab_size, embed_dim)
    
    # 词汇表到索引的映射
    word_to_idx = {word: idx for idx, word in enumerate(vocab)}
    
    return embedding, word_to_idx, vocab

# 创建词嵌入
embedding, word_to_idx, vocab = create_word_embeddings()

# 定义一个句子
sentence = ['the', 'big', 'cat', 'sat', 'on', 'the', 'small', 'mat']
print(f"原句子: {' '.join(sentence)}")

# 将句子转换为索引
try:
    sentence_indices = [word_to_idx[word] for word in sentence if word in word_to_idx]
    valid_sentence = [word for word in sentence if word in word_to_idx]
    print(f"有效句子: {' '.join(valid_sentence)}")
    print(f"词索引: {sentence_indices}")
except KeyError as e:
    print(f"词汇表中没有找到词: {e}")
    # 使用一个简化的句子
    valid_sentence = ['the', 'big', 'cat', 'sat', 'on', 'the']
    sentence_indices = [word_to_idx[word] for word in valid_sentence]
    print(f"使用简化句子: {' '.join(valid_sentence)}")
    print(f"词索引: {sentence_indices}")

# 获取词嵌入
sentence_tensor = torch.tensor(sentence_indices).unsqueeze(0)  # 添加batch维度
embedded_sentence = embedding(sentence_tensor)  # [1, seq_len, embed_dim]

print(f"\n嵌入后的句子形状: {embedded_sentence.shape}")

In [ ]:
# 对句子应用自注意力
embed_dim = embedded_sentence.size(-1)
sentence_self_attention = SelfAttention(embed_dim)

# 计算自注意力
attended_output, sentence_attention_weights = sentence_self_attention(embedded_sentence)

print(f"自注意力输出形状: {attended_output.shape}")
print(f"注意力权重形状: {sentence_attention_weights.shape}")

# 可视化句子的自注意力权重
visualize_self_attention(
    sentence_attention_weights, 
    tokens=valid_sentence,
    title=f"句子 '{' '.join(valid_sentence)}' 的自注意力权重"
)

## 2.5 分析注意力权重

让我们详细分析注意力权重，理解模型学到了什么：

In [ ]:
def analyze_attention_patterns(attention_weights, tokens):
    """
    分析注意力模式
    """
    # 取第一个batch的权重
    weights = attention_weights[0].detach().numpy()
    
    print("=== 注意力模式分析 ===")
    print(f"句子: {' '.join(tokens)}")
    print()
    
    # 找出每个位置最关注的其他位置
    for i, token in enumerate(tokens):
        # 排除自己，找到最大注意力权重
        other_weights = weights[i].copy()
        other_weights[i] = 0  # 排除自己
        max_idx = np.argmax(other_weights)
        max_weight = other_weights[max_idx]
        
        # 自己的注意力权重
        self_weight = weights[i][i]
        
        print(f"'{token}' (位置{i}):")
        print(f"  最关注: '{tokens[max_idx]}' (权重: {max_weight:.3f})")
        print(f"  自注意力: {self_weight:.3f}")
        print(f"  是否主要关注自己: {'是' if self_weight > max_weight else '否'}")
        print()

# 分析句子的注意力模式
analyze_attention_patterns(sentence_attention_weights, valid_sentence)

## 2.6 掩码注意力 (Masked Attention)

在某些应用中，我们需要限制注意力的范围。例如：
1. **填充掩码（Padding Mask）**：忽略填充位置
2. **因果掩码（Causal Mask）**：在语言模型中，防止看到未来的token

让我们实现和演示掩码注意力：

In [ ]:
def create_causal_mask(seq_len):
    """
    创建因果掩码（下三角矩阵）
    防止模型看到未来的token
    """
    mask = torch.tril(torch.ones(seq_len, seq_len))
    return mask.unsqueeze(0)  # 添加batch维度

def create_padding_mask(seq_len, valid_len):
    """
    创建填充掩码
    """
    mask = torch.zeros(1, seq_len, seq_len)
    mask[:, :valid_len, :valid_len] = 1
    return mask

# 演示因果掩码
seq_len = 6
causal_mask = create_causal_mask(seq_len)

print("因果掩码（下三角矩阵）:")
print(causal_mask[0].numpy())

# 可视化因果掩码
plt.figure(figsize=(8, 6))
sns.heatmap(causal_mask[0].numpy(), annot=True, cmap='Blues', 
            xticklabels=[f'位置{i}' for i in range(seq_len)],
            yticklabels=[f'位置{i}' for i in range(seq_len)])
plt.title('因果掩码（1=可见，0=不可见）')
plt.xlabel('被关注的位置')
plt.ylabel('查询位置')
plt.tight_layout()
plt.show()

In [ ]:
# 演示带掩码的自注意力
test_input = torch.randn(1, seq_len, 32)  # [batch_size, seq_len, d_model]
masked_self_attention = SelfAttention(32)

# 不使用掩码
output_no_mask, weights_no_mask = masked_self_attention(test_input)

# 使用因果掩码
output_causal, weights_causal = masked_self_attention(test_input, mask=causal_mask)

# 比较可视化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 无掩码的注意力权重
im1 = ax1.imshow(weights_no_mask[0].detach().numpy(), cmap='Blues')
ax1.set_title('无掩码的自注意力权重')
ax1.set_xlabel('被关注的位置')
ax1.set_ylabel('查询位置')

# 添加数值标注
for i in range(seq_len):
    for j in range(seq_len):
        ax1.text(j, i, f'{weights_no_mask[0][i][j]:.2f}', 
                ha='center', va='center', fontsize=8)

# 有掩码的注意力权重
im2 = ax2.imshow(weights_causal[0].detach().numpy(), cmap='Blues')
ax2.set_title('带因果掩码的自注意力权重')
ax2.set_xlabel('被关注的位置')
ax2.set_ylabel('查询位置')

# 添加数值标注
for i in range(seq_len):
    for j in range(seq_len):
        ax2.text(j, i, f'{weights_causal[0][i][j]:.2f}', 
                ha='center', va='center', fontsize=8)

plt.tight_layout()
plt.show()

print("观察：")
print("- 无掩码：每个位置都可以关注所有位置")
print("- 因果掩码：每个位置只能关注当前位置及之前的位置")
print("- 这在语言模型中很重要，防止模型"作弊"看到未来的词")

## 2.7 自注意力 vs 传统注意力

让我们总结一下自注意力和传统注意力的区别：

In [ ]:
# 创建对比表格
import pandas as pd

comparison_data = {
    '特性': ['Query来源', 'Key来源', 'Value来源', '主要用途', '优势', '典型应用'],
    '传统注意力': [
        '目标序列', 
        '源序列', 
        '源序列',
        '建立不同序列间的关系',
        '跨序列信息传递',
        '机器翻译、图像标注'
    ],
    '自注意力': [
        '同一序列',
        '同一序列', 
        '同一序列',
        '建立序列内部的关系',
        '捕获长距离依赖、并行计算',
        'Transformer、BERT、GPT'
    ]
}

df = pd.DataFrame(comparison_data)
print("传统注意力 vs 自注意力对比")
print("=" * 50)
print(df.to_string(index=False))

## 2.8 自注意力的计算复杂度分析

理解自注意力的计算复杂度对于实际应用很重要：

In [ ]:
def analyze_complexity():
    """
    分析不同序列长度下的计算复杂度
    """
    seq_lengths = [64, 128, 256, 512, 1024]
    d_model = 512
    
    print("自注意力计算复杂度分析")
    print("=" * 40)
    print(f"模型维度 d_model = {d_model}")
    print()
    print(f"{'序列长度':<10} {'QKV变换':<15} {'注意力计算':<15} {'总FLOPs':<15}")
    print("-" * 55)
    
    for n in seq_lengths:
        # QKV线性变换的FLOPs: 3 * n * d_model^2
        qkv_flops = 3 * n * d_model * d_model
        
        # 注意力计算的FLOPs: n^2 * d_model (for QK^T) + n^2 * d_model (for AV)
        attention_flops = 2 * n * n * d_model
        
        total_flops = qkv_flops + attention_flops
        
        print(f"{n:<10} {qkv_flops/1e6:<15.1f} {attention_flops/1e6:<15.1f} {total_flops/1e6:<15.1f}")
    
    print("\n注：FLOPs单位为百万次操作 (M)")
    print("观察：注意力计算的复杂度随序列长度的平方增长")
    
    # 绘制复杂度曲线
    plt.figure(figsize=(10, 6))
    
    qkv_complexity = [3 * n * d_model * d_model / 1e6 for n in seq_lengths]
    attention_complexity = [2 * n * n * d_model / 1e6 for n in seq_lengths]
    
    plt.plot(seq_lengths, qkv_complexity, 'b-o', label='QKV线性变换 O(n)')
    plt.plot(seq_lengths, attention_complexity, 'r-o', label='注意力计算 O(n²)')
    
    plt.xlabel('序列长度')
    plt.ylabel('计算量 (M FLOPs)')
    plt.title('自注意力计算复杂度分析')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.yscale('log')
    plt.tight_layout()
    plt.show()

analyze_complexity()

## 2.9 练习和实验

### 练习1: 实现不同的注意力变体

In [ ]:
class AdditiveAttention(nn.Module):
    """
    加性注意力机制（相对于点积注意力）
    """
    def __init__(self, d_model, d_k):
        super(AdditiveAttention, self).__init__()
        self.d_model = d_model
        self.d_k = d_k
        
        self.W_q = nn.Linear(d_model, d_k, bias=False)
        self.W_k = nn.Linear(d_model, d_k, bias=False)
        self.W_v = nn.Linear(d_model, d_k, bias=False)
        
        # 加性注意力的参数
        self.W_a = nn.Linear(d_k, 1, bias=False)
        self.W_o = nn.Linear(d_k, d_model)
        
    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        
        Q = self.W_q(x)  # [batch_size, seq_len, d_k]
        K = self.W_k(x)  # [batch_size, seq_len, d_k]
        V = self.W_v(x)  # [batch_size, seq_len, d_k]
        
        # 加性注意力：对每对(i,j)计算 W_a^T * tanh(Q_i + K_j)
        # 扩展维度进行广播
        Q_expanded = Q.unsqueeze(2)  # [batch_size, seq_len, 1, d_k]
        K_expanded = K.unsqueeze(1)  # [batch_size, 1, seq_len, d_k]
        
        # 计算Q + K
        combined = Q_expanded + K_expanded  # [batch_size, seq_len, seq_len, d_k]
        
        # 应用tanh和线性变换
        scores = self.W_a(torch.tanh(combined)).squeeze(-1)  # [batch_size, seq_len, seq_len]
        
        # Softmax
        attention_weights = F.softmax(scores, dim=-1)
        
        # 应用注意力
        context = torch.matmul(attention_weights, V)
        output = self.W_o(context)
        
        return output, attention_weights

# 比较点积注意力和加性注意力
d_model = 32
d_k = 32
seq_len = 6

x_test = torch.randn(1, seq_len, d_model)

# 点积注意力
dot_attention = SelfAttention(d_model, d_k)
dot_output, dot_weights = dot_attention(x_test)

# 加性注意力
add_attention = AdditiveAttention(d_model, d_k)
add_output, add_weights = add_attention(x_test)

# 可视化比较
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 点积注意力
im1 = ax1.imshow(dot_weights[0].detach().numpy(), cmap='Blues')
ax1.set_title('点积自注意力')
ax1.set_xlabel('Key位置')
ax1.set_ylabel('Query位置')

# 加性注意力
im2 = ax2.imshow(add_weights[0].detach().numpy(), cmap='Blues')
ax2.set_title('加性自注意力')
ax2.set_xlabel('Key位置')
ax2.set_ylabel('Query位置')

plt.tight_layout()
plt.show()

print("比较结果：")
print(f"点积注意力输出范围: [{dot_weights.min():.3f}, {dot_weights.max():.3f}]")
print(f"加性注意力输出范围: [{add_weights.min():.3f}, {add_weights.max():.3f}]")

### 练习2: 注意力头的可视化分析

In [ ]:
def attention_pattern_analysis(attention_weights, title="注意力模式分析"):
    """
    分析注意力权重的统计特性
    """
    weights = attention_weights[0].detach().numpy()
    
    # 计算统计指标
    diagonal_mean = np.mean(np.diag(weights))  # 对角线均值（自注意力）
    off_diagonal_mean = np.mean(weights - np.diag(np.diag(weights)))  # 非对角线均值
    entropy = -np.sum(weights * np.log(weights + 1e-9), axis=-1).mean()  # 注意力熵
    
    print(f"=== {title} ===")
    print(f"对角线注意力均值（自注意力）: {diagonal_mean:.3f}")
    print(f"非对角线注意力均值: {off_diagonal_mean:.3f}")
    print(f"注意力熵（分散程度）: {entropy:.3f}")
    print(f"是否主要关注自己: {'是' if diagonal_mean > off_diagonal_mean else '否'}")
    print()
    
    # 可视化注意力分布
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))
    
    # 1. 注意力热力图
    sns.heatmap(weights, annot=True, fmt='.2f', cmap='Blues', ax=ax1)
    ax1.set_title('注意力权重矩阵')
    
    # 2. 对角线 vs 非对角线
    diagonal_vals = np.diag(weights)
    positions = range(len(diagonal_vals))
    ax2.bar(positions, diagonal_vals, alpha=0.7, label='自注意力')
    ax2.set_xlabel('位置')
    ax2.set_ylabel('注意力权重')
    ax2.set_title('各位置的自注意力权重')
    ax2.legend()
    
    # 3. 注意力熵分布
    row_entropies = -np.sum(weights * np.log(weights + 1e-9), axis=-1)
    ax3.bar(positions, row_entropies, alpha=0.7, color='orange')
    ax3.set_xlabel('查询位置')
    ax3.set_ylabel('注意力熵')
    ax3.set_title('各位置的注意力分散程度')
    
    # 4. 注意力权重分布直方图
    ax4.hist(weights.flatten(), bins=20, alpha=0.7, color='green')
    ax4.set_xlabel('注意力权重值')
    ax4.set_ylabel('频次')
    ax4.set_title('注意力权重分布')
    
    plt.tight_layout()
    plt.show()

# 分析之前计算的注意力权重
attention_pattern_analysis(dot_weights, "点积自注意力模式分析")
attention_pattern_analysis(add_weights, "加性自注意力模式分析")

## 总结

在这个教程中，我们深入学习了自注意力机制：

### 关键概念：
1. **自注意力定义**：Query、Key、Value都来自同一序列
2. **数学公式**：$\text{SelfAttention}(X) = \text{softmax}(\frac{QK^T}{\sqrt{d_k}})V$
3. **掩码机制**：控制注意力的可见范围
4. **计算复杂度**：$O(n^2 \cdot d)$，其中n是序列长度

### 重要特性：
- **并行计算**：所有位置可以同时计算
- **长距离依赖**：任意位置间距离为常数
- **可解释性**：注意力权重提供直观理解
- **灵活性**：可以通过掩码控制注意力范围

### 实际应用：
- **语言建模**：使用因果掩码防止看到未来
- **文本理解**：捕获词语间的语义关系
- **序列标注**：利用全局上下文信息

自注意力是Transformer架构的基础。在下一个教程中，我们将学习**多头注意力机制（Multi-Head Attention）**，它进一步提升了自注意力的表达能力。

### 下一步学习：
- [03-multi-head-attention.ipynb](03-multi-head-attention.ipynb) - 多头注意力机制详解